In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn.metrics import roc_auc_score
import pandas as pd
import numpy as np
# from imblearn.over_sampling import SMOTE

from pyexplainer.pyexplainer_pyexplainer import PyExplainer
import matplotlib.pyplot as plt

import os, pickle, time
# from datetime import datetime

from my_util import *

from lime.lime.lime_tabular import LimeTabularExplainer

In [14]:
proj_name = 'openstack'
dump_dataframe_dir = './dump_df/'
x_train, x_test = prepare_data(proj_name, mode = 'train')
global_model = pickle.load(open(proj_name+'_global_model.pkl','rb'))

correctly_predict_df = pd.read_csv(dump_dataframe_dir+proj_name+'_correctly_predict_as_defective.csv')
correctly_predict_df = correctly_predict_df.set_index('commit_id')

class_label = ['clean', 'defect']
dep = 'defect'
indep = correctly_predict_df.columns[:-3] # exclude the last 3 columns
new_correctly_predict_df = correctly_predict_df.loc[:, indep] 

In [3]:
print(list(x_train.columns).index('self'))
categorical_features_list = [6]

6


In [4]:
# print(new_correctly_predict_df)


In [5]:
lime_explainer = LimeTabularExplainer(x_train.values, categorical_features=categorical_features_list, 
                                      feature_names=indep, class_names=class_label, 
                                      random_state=0)

In [6]:
sample_instance = new_correctly_predict_df.iloc[0]
# note: must use synt_inst_for_local_model with local model
# note: use synt_inst to find distance with original instance
exp, synt_inst, synt_inst_for_local_model, local_model = lime_explainer.explain_instance(sample_instance, global_model.predict_proba)

explain finished


In [18]:
print(new_correctly_predict_df.iloc[[0]].values)

[[2.58000000e+02 2.00000000e+00 1.00000000e+00 8.26278101e-01
  2.00000000e+00 1.80495602e+01 0.00000000e+00 1.00000000e+00
  5.24275463e+00 4.00000000e+00 4.65000000e+02 4.26212040e-03
  8.20458178e-02]]


In [12]:
print(synt_inst_for_local)

[[1. 1. 1. ... 1. 1. 1.]
 [0. 1. 1. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 1.]
 ...
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 1. 0. 1.]
 [1. 1. 1. ... 0. 0. 0.]]


In [8]:
# exp.show_in_notebook()

In [9]:
exp.as_list() # get rule from LIME

[('la > 58.00', 0.17364610728196841),
 ('0.63 < ent <= 0.86', 0.10334290441719379),
 ('ns <= 1.00', 0.08326492305826197),
 ('1.00 < nrev <= 2.00', -0.03680321984702016),
 ('self=0', 0.02505368474531611),
 ('asawr <= 0.01', -0.0232325371500721),
 ('rsawr <= 0.29', -0.021452708692225483),
 ('2.98 < age <= 9.98', 0.019697083991513934),
 ('1.00 < nd <= 2.00', -0.018826231313993257),
 ('app > 3.00', -0.01568265001489225)]

In [56]:
# eval('1.00 < nrev <= 2.00', {'nrev':1.35})
the_best_rule = exp.as_list()[0][0] # get the best rule
the_best_rule = 'ent <= 0.9050000011920929 & rtime > 1.6699999570846558 & nd <= 2.9850000143051147'
# print(the_best_rule)
var_in_rule = re.findall('[a-zA-Z]+',the_best_rule) # extract variable from rules
val_of_var = 1.58 # get value from df (done later...)
print(the_best_rule)
print(var_in_rule)
# print(eval(the_best_rule, {var_in_rule: val_of_var})) # then check condition

ent <= 0.9050000011920929 & rtime > 1.6699999570846558 & nd <= 2.9850000143051147
['ent', 'rtime', 'nd']


In [19]:
exp.as_map()

{1: [(0, 0.17364610728196841),
  (3, 0.10334290441719379),
  (2, 0.08326492305826197),
  (4, -0.03680321984702016),
  (6, 0.02505368474531611),
  (11, -0.0232325371500721),
  (12, -0.021452708692225483),
  (8, 0.019697083991513934),
  (1, -0.018826231313993257),
  (9, -0.01568265001489225)]}

In [26]:
exp.local_exp[1]

[(0, 0.17364610728196841),
 (3, 0.10334290441719379),
 (2, 0.08326492305826197),
 (4, -0.03680321984702016),
 (6, 0.02505368474531611),
 (11, -0.0232325371500721),
 (12, -0.021452708692225483),
 (8, 0.019697083991513934),
 (1, -0.018826231313993257),
 (9, -0.01568265001489225)]